In [18]:

import os
import sys
import logging
import itertools
import warnings
import argparse
import json
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd
import xarray as xr
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.feature_selection import mutual_info_regression
import joblib

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)
logger = logging.getLogger(__name__)

# ========================================
# CONFIGURATION
# ========================================
@dataclass
class Config:
    """Configuration parameters for the UAV wind prediction pipeline."""
    grib_path: Path = Path(r"C:\Users\Victus\Downloads\953c78511a142e4b81be9c53038b62a5.grib")
    pressure_levels: Tuple[int, ...] = (1000, 950, 900, 850)  # hPa, sorted high to low
    g: float = 9.80665  # Gravity constant (m/s^2)
    
    # Data processing optimization
    spatial_stride: int = 2        # Keep every 2nd lat/lon point
    temporal_stride: int = 6       # Keep every 6th hour
    
    # Model parameters
    test_ratio: float = 0.20
    random_state: int = 42
    n_estimators: int = 200
    max_depth: int = 15
    min_samples_split: int = 10
    min_samples_leaf: int = 5
    
    # Output paths
    models_dir: Path = Path("./models")
    logs_dir: Path = Path("./logs")
    
    # Feature engineering options
    enable_lstm: bool = False
    sequence_length: int = 24
    hyperparameter_tuning: bool = False

# Global configuration instance
CFG = Config()

# ========================================
# ENHANCED FEATURE ENGINEERING
# ========================================
def engineer_features_comprehensive(ds: xr.Dataset) -> xr.Dataset:
    """
    Comprehensive feature engineering with all atmospheric relationships.
    Processes all levels at once for efficiency.
    """
    logger.info("Starting comprehensive feature engineering...")
    
    # Ensure we have the required pressure levels
    available_levels = sorted([int(p) for p in ds.isobaricInhPa.values], reverse=True)
    target_levels = [p for p in CFG.pressure_levels if p in available_levels]
    
    if len(target_levels) != len(CFG.pressure_levels):
        logger.warning(f"Only {len(target_levels)} of {len(CFG.pressure_levels)} levels available")
    
    # Select required pressure levels
    ds = ds.sel(isobaricInhPa=target_levels)
    
    # Calculate base wind speed for all levels
    ds["wind_speed"] = np.hypot(ds.u, ds.v)
    
    logger.info(f"Engineering features for {len(target_levels)} pressure levels...")
    
    # ===== LEVEL-SPECIFIC FEATURES =====
    for p in target_levels:
        # Primary wind variables
        ds[f"ws{p}"] = ds.wind_speed.sel(isobaricInhPa=p)
        ds[f"u{p}"] = ds.u.sel(isobaricInhPa=p)
        ds[f"v{p}"] = ds.v.sel(isobaricInhPa=p)
        
        # Wind direction (meteorological convention)
        ang = np.arctan2(ds.v.sel(isobaricInhPa=p), ds.u.sel(isobaricInhPa=p))
        ds[f"wd{p}"] = (270 - np.degrees(ang)) % 360
        
        # Temperature (K to °C)
        ds[f"temp{p}"] = ds.t.sel(isobaricInhPa=p) - 273.15
        
        # Geopotential height (m)
        ds[f"height{p}"] = ds.z.sel(isobaricInhPa=p) / CFG.g
        
        # Optional atmospheric variables (if available)
        optional_vars = {
            'w': f'w{p}',           # Vertical velocity
            'pv': f'pv{p}',         # Potential vorticity
            'vo': f'vort{p}',       # Relative vorticity
            'd': f'div{p}',         # Divergence
            'q': f'q{p}',           # Specific humidity
            'r': f'r{p}'            # Relative humidity
        }
        
        for var, name in optional_vars.items():
            if var in ds.data_vars:
                ds[name] = ds[var].sel(isobaricInhPa=p)
    
    # ===== INTER-LEVEL FEATURES =====
    # Compute relationships between all level pairs
    level_pairs = [(b, t) for b, t in itertools.combinations(target_levels, 2) if b > t]
    logger.info(f"Computing inter-level features for {len(level_pairs)} level pairs")
    
    for bottom, top in level_pairs:
        # Wind shear magnitude (vector difference)
        du = ds[f"u{top}"] - ds[f"u{bottom}"]
        dv = ds[f"v{top}"] - ds[f"v{bottom}"]
        ds[f"shear_{bottom}_{top}"] = np.hypot(du, dv)
        
        # Directional wind shear
        ang_bottom = np.arctan2(ds[f"v{bottom}"], ds[f"u{bottom}"])
        ang_top = np.arctan2(ds[f"v{top}"], ds[f"u{top}"])
        ddir = ang_top - ang_bottom
        # Handle angle wrapping
        ddir = np.where(ddir > np.pi, ddir - 2 * np.pi, ddir)
        ddir = np.where(ddir < -np.pi, ddir + 2 * np.pi, ddir)
        ds[f"dirshear_{bottom}_{top}"] = (("time", "latitude", "longitude"), ddir)
        
        # Temperature lapse rate (°C/km)
        dz = ds[f"height{top}"] - ds[f"height{bottom}"]
        dt = ds[f"temp{bottom}"] - ds[f"temp{top}"]
        ds[f"lapse_{bottom}_{top}"] = (dt / dz) * 1000  # Convert to °C/km
        
        # Height and temperature differences
        ds[f"dz_{bottom}_{top}"] = dz
        ds[f"dt_{bottom}_{top}"] = dt
        
        # Wind speed differences
        ds[f"dws_{bottom}_{top}"] = ds[f"ws{top}"] - ds[f"ws{bottom}"]
    
    # ===== ATMOSPHERIC COLUMN FEATURES =====
    # Mean properties across all levels
    ds["temp_column_mean"] = ds.t.mean("isobaricInhPa") - 273.15
    ds["ws_column_mean"] = ds.wind_speed.mean("isobaricInhPa")
    ds["u_column_mean"] = ds.u.mean("isobaricInhPa")
    ds["v_column_mean"] = ds.v.mean("isobaricInhPa")
    
    # Atmospheric stability indicators
    if len(target_levels) >= 2:
        # Bulk Richardson number approximation
        surface_level = max(target_levels)
        upper_level = min(target_levels)
        
        dtheta = (ds[f"temp{upper_level}"] - ds[f"temp{surface_level}"]) + \
                 (CFG.g / 1004) * (ds[f"height{upper_level}"] - ds[f"height{surface_level}"])
        du_bulk = ds[f"u{upper_level}"] - ds[f"u{surface_level}"]
        dv_bulk = ds[f"v{upper_level}"] - ds[f"v{surface_level}"]
        wind_shear_bulk = np.hypot(du_bulk, dv_bulk)
        
        ds["bulk_richardson"] = (CFG.g * dtheta * (ds[f"height{upper_level}"] - ds[f"height{surface_level}"])) / \
                               (ds[f"temp{surface_level}"] * wind_shear_bulk**2 + 1e-10)
    
    # Column-integrated features (if humidity available)
    if 'q' in ds.data_vars:
        ds["q_column_mean"] = ds.q.mean("isobaricInhPa")
        # Precipitable water approximation
        dp_levels = np.diff(sorted(target_levels, reverse=True))
        if len(dp_levels) > 0:
            ds["precipitable_water"] = (ds.q * np.mean(dp_levels) * 100 / CFG.g).sum("isobaricInhPa")
    
    if 'r' in ds.data_vars:
        ds["rh_column_mean"] = ds.r.mean("isobaricInhPa")
    
    # ===== GEOGRAPHIC AND TEMPORAL FEATURES =====
    # Geographic coordinates
    ds["lat"] = ds.latitude
    ds["lon"] = ds.longitude
    
    # Terrain approximation (using surface geopotential if available)
    if len(target_levels) > 0:
        surface_level = max(target_levels)
        ds["terrain_height"] = ds[f"height{surface_level}"]
    
    # Time-based features with cyclic encoding
    if 'time' in ds.dims:
        time_vals = pd.to_datetime(ds.time.values)
        
        # Basic time features
        ds["hour"] = (("time",), time_vals.hour.values)
        ds["day_of_year"] = (("time",), time_vals.dayofyear.values)
        ds["month"] = (("time",), time_vals.month.values)
        
        # Cyclic encodings
        ds["hour_sin"] = np.sin(2 * np.pi * ds["hour"] / 24)
        ds["hour_cos"] = np.cos(2 * np.pi * ds["hour"] / 24)
        ds["doy_sin"] = np.sin(2 * np.pi * ds["day_of_year"] / 365)
        ds["doy_cos"] = np.cos(2 * np.pi * ds["day_of_year"] / 365)
        ds["month_sin"] = np.sin(2 * np.pi * ds["month"] / 12)
        ds["month_cos"] = np.cos(2 * np.pi * ds["month"] / 12)
    
    logger.info(f"Feature engineering complete. Total variables: {len(ds.data_vars)}")
    return ds

def get_features_excluding_target(ds: xr.Dataset, target_level: int) -> List[str]:
    """
    Get feature names while excluding target level to prevent data leakage.
    This is the key leakage prevention mechanism from Script 1.
    """
    # Variables to always exclude
    exclude_base = {'wind_speed', 'u', 'v', 't', 'z', 'time', 'step', 'surface', 
                   'latitude', 'longitude', 'isobaricInhPa'}
    
    # Target variable name
    target_var = f"ws{target_level}"
    
    # All variables that contain the target level (to prevent leakage)
    level_specific_exclusions = {var for var in ds.data_vars.keys() 
                                if str(target_level) in var and var != target_var}
    
    # Combine all exclusions
    all_exclusions = exclude_base | level_specific_exclusions | {target_var}
    
    # Get valid features
    valid_features = [var for var in ds.data_vars.keys() if var not in all_exclusions]
    
    logger.info(f"Target level {target_level} hPa: {len(valid_features)} features "
               f"(excluded {len(level_specific_exclusions)} level-specific vars)")
    
    return valid_features

# ========================================
# EFFICIENT DATA PROCESSING
# ========================================
def apply_spatial_temporal_thinning(ds: xr.Dataset) -> xr.Dataset:
    """Apply spatial and temporal thinning for computational efficiency."""
    logger.info(f"Applying thinning: spatial stride={CFG.spatial_stride}, "
               f"temporal stride={CFG.temporal_stride}")
    
    # Spatial thinning
    if CFG.spatial_stride > 1:
        lat_indices = slice(None, None, CFG.spatial_stride)
        lon_indices = slice(None, None, CFG.spatial_stride)
        ds = ds.isel(latitude=lat_indices, longitude=lon_indices)
    
    # Temporal thinning
    if CFG.temporal_stride > 1 and 'time' in ds.dims:
        time_indices = slice(None, None, CFG.temporal_stride)
        ds = ds.isel(time=time_indices)
    
    logger.info(f"Grid after thinning: {dict(ds.dims)}")
    return ds

def to_dataframe_optimized(ds: xr.Dataset) -> pd.DataFrame:
    """Convert xarray to pandas with memory optimization."""
    logger.info("Converting to DataFrame with optimization...")
    
    # Apply thinning first
    ds_thin = apply_spatial_temporal_thinning(ds)
    
    # Convert to DataFrame
    df = ds_thin.to_dataframe().dropna().reset_index()
    
    logger.info(f"DataFrame shape after optimization: {df.shape}")
    return df

# ========================================
# ENHANCED MODEL TRAINING
# ========================================
def temporal_train_test_split(df: pd.DataFrame, test_ratio: float = 0.2) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Temporal split to prevent data leakage in time series.
    """
    if 'time' in df.columns:
        df_sorted = df.sort_values('time')
        split_idx = int(len(df_sorted) * (1 - test_ratio))
        train_df = df_sorted.iloc[:split_idx]
        test_df = df_sorted.iloc[split_idx:]
        logger.info(f"Temporal split: {len(train_df):,} train, {len(test_df):,} test samples")
        return train_df, test_df
    else:
        logger.warning("No time column found, using random split")
        from sklearn.model_selection import train_test_split
        return train_test_split(df, test_size=test_ratio, random_state=CFG.random_state)

def hyperparameter_optimization(X_train: np.ndarray, y_train: np.ndarray) -> Dict[str, Any]:
    """
    Perform hyperparameter optimization using RandomizedSearchCV.
    """
    logger.info("Starting hyperparameter optimization...")
    
    param_distributions = {
        'n_estimators': [100, 150, 200, 250],
        'max_depth': [10, 15, 20, 25, None],
        'min_samples_split': [5, 10, 15, 20],
        'min_samples_leaf': [2, 5, 10],
        'max_features': ['sqrt', 'log2', None]
    }
    
    rf_base = RandomForestRegressor(random_state=CFG.random_state, n_jobs=-1, oob_score=True)
    
    # Use TimeSeriesSplit for cross-validation
    tscv = TimeSeriesSplit(n_splits=3)
    
    search = RandomizedSearchCV(
        rf_base, 
        param_distributions,
        n_iter=50,
        cv=tscv,
        scoring='neg_mean_squared_error',
        n_jobs=-1,
        random_state=CFG.random_state,
        verbose=1
    )
    
    search.fit(X_train, y_train)
    
    logger.info(f"Best hyperparameters: {search.best_params_}")
    logger.info(f"Best CV score: {-search.best_score_:.4f}")
    
    return search.best_params_

def train_enhanced_model(df: pd.DataFrame, target_level: int) -> Dict[str, Any]:
    """
    Train an enhanced model for a specific pressure level.
    """
    logger.info(f"Training model for {target_level} hPa...")
    
    # Get features (excluding target level to prevent leakage)
    # We need to reconstruct the dataset to get proper feature exclusion
    target_var = f"ws{target_level}"
    
    # Get all features except those containing the target level
    exclude_patterns = {target_var, 'wind_speed', 'u', 'v', 't', 'z'}
    exclude_with_level = [col for col in df.columns if str(target_level) in col and col != target_var]
    all_exclusions = exclude_patterns | set(exclude_with_level)
    
    feature_cols = [col for col in df.columns if col not in all_exclusions]
    
    # Prepare data
    df_model = df[feature_cols + [target_var]].copy()
    
    # Temporal split
    train_df, test_df = temporal_train_test_split(df_model, CFG.test_ratio)
    
    X_train = train_df[feature_cols]
    y_train = train_df[target_var]
    X_test = test_df[feature_cols]
    y_test = test_df[target_var]
    
    # Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Hyperparameter optimization (if enabled)
    if CFG.hyperparameter_tuning:
        best_params = hyperparameter_optimization(X_train_scaled, y_train)
    else:
        best_params = {
            'n_estimators': CFG.n_estimators,
            'max_depth': CFG.max_depth,
            'min_samples_split': CFG.min_samples_split,
            'min_samples_leaf': CFG.min_samples_leaf
        }
    
    # Train final model
    rf_model = RandomForestRegressor(
        **best_params,
        oob_score=True,
        n_jobs=-1,
        random_state=CFG.random_state
    )
    
    rf_model.fit(X_train_scaled, y_train)
    
    # Evaluate model
    y_pred_train = rf_model.predict(X_train_scaled)
    y_pred_test = rf_model.predict(X_test_scaled)
    
    performance = {
        'train_rmse': np.sqrt(mean_squared_error(y_train, y_pred_train)),
        'test_rmse': np.sqrt(mean_squared_error(y_test, y_pred_test)),
        'train_r2': r2_score(y_train, y_pred_train),
        'test_r2': r2_score(y_test, y_pred_test),
        'train_mae': mean_absolute_error(y_train, y_pred_train),
        'test_mae': mean_absolute_error(y_test, y_pred_test),
        'oob_r2': rf_model.oob_score_
    }
    
    # Check for overfitting
    overfitting_gap = performance['train_r2'] - performance['test_r2']
    performance['overfitting_detected'] = overfitting_gap > 0.1
    
    logger.info(f"Level {target_level} hPa - Test R²: {performance['test_r2']:.3f}, "
               f"Test RMSE: {performance['test_rmse']:.3f} m/s")
    
    if performance['overfitting_detected']:
        logger.warning(f"Possible overfitting detected (R² gap: {overfitting_gap:.3f})")
    
    # Feature importance analysis
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # Save model artifacts
    CFG.models_dir.mkdir(exist_ok=True)
    model_path = CFG.models_dir / f"rf_{target_level}hPa.pkl"
    scaler_path = CFG.models_dir / f"scaler_{target_level}hPa.pkl"
    
    joblib.dump(rf_model, model_path, compress=3)
    joblib.dump(scaler, scaler_path, compress=3)
    
    logger.info(f"Saved model: {model_path}")
    logger.info(f"Saved scaler: {scaler_path}")
    
    return {
        'model': rf_model,
        'scaler': scaler,
        'features': feature_cols,
        'performance': performance,
        'feature_importance': feature_importance,
        'hyperparameters': best_params
    }

# ========================================
# MAIN TRAINING PIPELINE
# ========================================
def train_all_models() -> Dict[int, Dict[str, Any]]:
    """
    Train models for all pressure levels with enhanced features.
    """
    logger.info("="*60)
    logger.info("ENHANCED UAV WIND PREDICTION PIPELINE")
    logger.info("="*60)
    logger.info(f"Training models for pressure levels: {CFG.pressure_levels} hPa")
    logger.info(f"Data source: {CFG.grib_path}")
    
    # Load and process data
    logger.info("Loading GRIB data...")
    ds_raw = xr.open_dataset(CFG.grib_path, engine="cfgrib")
    logger.info(f"Raw data dimensions: {dict(ds_raw.dims)}")
    
    # Feature engineering
    ds_features = engineer_features_comprehensive(ds_raw)
    
    # Convert to DataFrame with optimization
    df = to_dataframe_optimized(ds_features)
    
    # Train models for each level
    trained_models = {}
    
    for target_level in CFG.pressure_levels:
        logger.info("="*50)
        logger.info(f"TRAINING MODEL FOR {target_level} hPa")
        logger.info("="*50)
        
        try:
            model_info = train_enhanced_model(df, target_level)
            trained_models[target_level] = model_info
        except Exception as e:
            logger.error(f"Failed to train model for {target_level} hPa: {e}")
            continue
    
    # Save summary
    summary = {}
    for level, info in trained_models.items():
        summary[level] = info['performance']
        summary[level]['hyperparameters'] = info['hyperparameters']
    
    summary_path = CFG.models_dir / "training_summary.json"
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2, default=str)
    
    logger.info(f"Training summary saved: {summary_path}")
    
    # Print final summary
    logger.info("="*60)
    logger.info("TRAINING COMPLETED!")
    logger.info("="*60)
    logger.info(f"Successfully trained {len(trained_models)} models")
    
    print("\n📊 PERFORMANCE SUMMARY:")
    print(f"{'Level (hPa)':<12} {'Test R²':<10} {'Test RMSE':<12} {'Overfitting':<12}")
    print("-" * 50)
    
    for level, info in trained_models.items():
        perf = info['performance']
        status = "⚠️ High" if perf['overfitting_detected'] else "✅ Low"
        print(f"{level:<12} {perf['test_r2']:<10.3f} {perf['test_rmse']:<12.3f} {status}")
    
    return trained_models

# ========================================
# ALTITUDE TO PRESSURE LEVEL MAPPING
# ========================================
def altitude_to_pressure_level(altitude_m: float) -> int:
    """
    Convert altitude in meters to appropriate pressure level.
    Uses standard atmosphere approximation.
    """
    if altitude_m < 100:
        return 1000
    elif altitude_m < 600:
        return 950
    elif altitude_m < 1200:
        return 900
    else:
        return 850

# ========================================
# PREDICTION FUNCTIONS
# ========================================
def load_trained_models() -> Dict[int, Dict[str, Any]]:
    """Load all trained models from disk."""
    models = {}
    
    for level in CFG.pressure_levels:
        model_path = CFG.models_dir / f"rf_{level}hPa.pkl"
        scaler_path = CFG.models_dir / f"scaler_{level}hPa.pkl"
        
        if model_path.exists() and scaler_path.exists():
            models[level] = {
                'model': joblib.load(model_path),
                'scaler': joblib.load(scaler_path)
            }
            logger.info(f"Loaded model for {level} hPa")
        else:
            logger.warning(f"Model files not found for {level} hPa")
    
    return models

def predict_wind_along_path(path_points: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Predict wind speeds along a UAV flight path.
    
    Args:
        path_points: List of dictionaries with keys:
            - 'time': timestamp
            - 'latitude': latitude in degrees
            - 'longitude': longitude in degrees  
            - 'altitude_m': altitude in meters
    
    Returns:
        List of predictions with wind speed estimates
    """
    logger.info(f"Predicting wind for {len(path_points)} path points...")
    
    # Load models
    models = load_trained_models()
    
    if not models:
        logger.error("No trained models available")
        return []
    
    predictions = []
    
    for point in path_points:
        # Determine appropriate pressure level
        target_level = altitude_to_pressure_level(point['altitude_m'])
        
        if target_level not in models:
            logger.warning(f"No model available for pressure level {target_level} hPa")
            predictions.append({
                **point,
                'pressure_level': target_level,
                'wind_speed_ms': None,
                'prediction_status': 'no_model'
            })
            continue
        
        # In a real implementation, you would:
        # 1. Load weather data for the point's time/location
        # 2. Engineer features using the same process
        # 3. Apply the appropriate model
        
        # For now, return structure with placeholder
        predictions.append({
            **point,
            'pressure_level': target_level,
            'wind_speed_ms': None,  # Would be actual prediction
            'prediction_status': 'model_available'
        })
    
    return predictions

# ========================================
# COMMAND LINE INTERFACE
# ========================================
def create_cli() -> argparse.ArgumentParser:
    """Create command line interface."""
    parser = argparse.ArgumentParser(
        description="Enhanced UAV Wind Prediction Pipeline",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    
    # Main commands
    parser.add_argument("--train", action="store_true", 
                       help="Train models for all pressure levels")
    parser.add_argument("--predict-demo", action="store_true",
                       help="Run a demonstration prediction")
    
    # Configuration options
    parser.add_argument("--grib-path", type=Path, default=CFG.grib_path,
                       help="Path to GRIB data file")
    parser.add_argument("--models-dir", type=Path, default=CFG.models_dir,
                       help="Directory to save/load models")
    parser.add_argument("--spatial-stride", type=int, default=CFG.spatial_stride,
                       help="Spatial thinning stride")
    parser.add_argument("--temporal-stride", type=int, default=CFG.temporal_stride,
                       help="Temporal thinning stride")
    parser.add_argument("--hyperparameter-tuning", action="store_true",
                       help="Enable hyperparameter optimization")
    
    return parser

# ========================================
# MAIN ENTRY POINT
# ========================================
def main() -> None:
    """Parse CLI, update configuration, and run selected command."""
    parser = create_cli()

    # tolerant parse → unknown flags (e.g. from Jupyter) won't abort
    args, unknown = parser.parse_known_args()
    if unknown:
        logger.debug("Ignored unknown CLI flags: %s", unknown)

    # ── update runtime configuration ──────────────────────────────
    CFG.grib_path              = args.grib_path
    CFG.models_dir             = args.models_dir
    CFG.spatial_stride         = args.spatial_stride
    CFG.temporal_stride        = args.temporal_stride
    CFG.hyperparameter_tuning  = args.hyperparameter_tuning

    CFG.models_dir.mkdir(parents=True, exist_ok=True)
    CFG.logs_dir.mkdir(parents=True, exist_ok=True)

    # ── command routing ───────────────────────────────────────────
    if args.train:
        logger.info("▶ TRAIN selected ─ starting model training…")
        train_all_models()
        logger.info("✓ Training finished")

    elif args.predict_demo:
        logger.info("▶ PREDICT-DEMO selected ─ running demo…")
        demo_path = [
            dict(time=pd.Timestamp('2023-01-01 12:00'),
                 latitude=25.0, longitude=77.0, altitude_m=120),
            dict(time=pd.Timestamp('2023-01-01 13:00'),
                 latitude=25.1, longitude=77.1, altitude_m=200)
        ]
        preds = predict_wind_along_path(demo_path)

        print("\n🎯 DEMO PREDICTIONS")
        for p in preds:
            print(f" lat={p['latitude']:.2f}, lon={p['longitude']:.2f}, "
                  f"alt={p['altitude_m']} m  →  "
                  f"{p['pressure_level']} hPa | {p['prediction_status']} | "
                  f"wind={p['wind_speed_ms']}")
        print()

    else:
        # parser.print_help()
        logger.info("No command flag supplied → defaulting to --train")
        train_all_models()



# ------------------------------------------------
# standard entry point
# ------------------------------------------------
if __name__ == "__main__":
    main()


2025-06-29 20:31:39 | INFO | No command flag supplied → defaulting to --train
2025-06-29 20:31:39 | INFO | ============================================================
2025-06-29 20:31:39 | INFO | ENHANCED UAV WIND PREDICTION PIPELINE
2025-06-29 20:31:39 | INFO | ============================================================
2025-06-29 20:31:39 | INFO | Training models for pressure levels: (1000, 950, 900, 850) hPa
2025-06-29 20:31:39 | INFO | Data source: C:\Users\Victus\Downloads\953c78511a142e4b81be9c53038b62a5.grib
2025-06-29 20:31:39 | INFO | Loading GRIB data...
2025-06-29 20:31:39 | WARNING | Ignoring index file 'C:\\Users\\Victus\\Downloads\\953c78511a142e4b81be9c53038b62a5.grib.5b7b6.idx' incompatible with GRIB file
2025-06-29 20:32:55 | INFO | Raw data dimensions: {'time': 1464, 'isobaricInhPa': 4, 'latitude': 15, 'longitude': 17}
2025-06-29 20:32:55 | INFO | Starting comprehensive feature engineering...
2025-06-29 20:33:04 | INFO | Engineering features for 4 pressure levels...


📊 PERFORMANCE SUMMARY:
Level (hPa)  Test R²    Test RMSE    Overfitting 
--------------------------------------------------


In [1]:
import itertools
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import mutual_info_regression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import joblib
import warnings
warnings.filterwarnings('ignore')

# ========================================
# CONSTANTS AND CONFIGURATION
# ========================================
GRIB_PATH = "C:/Users/Victus/Downloads/953c78511a142e4b81be9c53038b62a5.grib"
PRESSURE_LEVELS = [1000, 950, 900, 850]  # hPa, sorted high to low
G = 9.80665  # Gravity (m/s^2)
MODELS_TO_TRAIN = PRESSURE_LEVELS  # We'll train models for all levels

print("="*60)
print("UAV WIND PREDICTION PIPELINE - MULTI-LEVEL APPROACH")
print("="*60)
print(f"Training models for pressure levels: {MODELS_TO_TRAIN} hPa")
print(f"Data source: {GRIB_PATH}")

# ========================================
# STEP 1: FEATURE ENGINEERING (NO LEAKAGE)
# ========================================
def engineer_features_safe(ds: xr.Dataset, target_level: int) -> xr.Dataset:
    """
    Engineer meteorological features WITHOUT data leakage for a specific target level.
    
    Key principle: NEVER use information from the target pressure level when 
    predicting that level's wind speed.
    """
    print(f"  → Engineering features for {target_level} hPa target (excluding {target_level} hPa data)")
    
    # Calculate wind speed for all levels first
    ds["wind_speed"] = np.hypot(ds.u, ds.v)
    
    # Get available levels (excluding target to prevent leakage)
    feature_levels = [p for p in PRESSURE_LEVELS if p != target_level]
    print(f"  → Using feature levels: {feature_levels}")
    
    # ===== LEVEL-SPECIFIC FEATURES (excluding target level) =====
    for p in feature_levels:
        # Wind speed and components
        ds[f"ws{p}"] = ds.wind_speed.sel(isobaricInhPa=p)
        ds[f"u{p}"] = ds.u.sel(isobaricInhPa=p)
        ds[f"v{p}"] = ds.v.sel(isobaricInhPa=p)
        
        # Wind direction
        ang = np.arctan2(ds.v.sel(isobaricInhPa=p), ds.u.sel(isobaricInhPa=p))
        ds[f"wd{p}"] = (270 - np.degrees(ang)) % 360
        
        # Temperature (K to °C)
        ds[f"temp{p}"] = ds.t.sel(isobaricInhPa=p) - 273.15
        
        # Geopotential height
        ds[f"height{p}"] = ds.z.sel(isobaricInhPa=p) / G
        
        # Optional atmospheric variables (if available)
        for var, name in [('w', 'w'), ('pv', 'pv'), ('vo', 'vort'), 
                         ('d', 'div'), ('q', 'q'), ('r', 'r')]:
            if var in ds.data_vars:
                ds[f"{name}{p}"] = ds[var].sel(isobaricInhPa=p)
    
    # ===== INTER-LEVEL FEATURES (excluding target level) =====
    # Only compute between non-target levels
    pairs = [(b, t) for b, t in itertools.combinations(feature_levels, 2) if b > t]
    print(f"  → Computing inter-level features for {len(pairs)} level pairs")
    
    for b, t in pairs:
        # Wind shear magnitude
        du = ds[f"u{t}"] - ds[f"u{b}"]
        dv = ds[f"v{t}"] - ds[f"v{b}"]
        ds[f"shear_{b}_{t}"] = np.hypot(du, dv)
        
        # Directional shear
        ang_b = np.arctan2(ds[f"v{b}"], ds[f"u{b}"])
        ang_t = np.arctan2(ds[f"v{t}"], ds[f"u{t}"])
        ddir = ang_t - ang_b
        ddir = np.where(ddir > np.pi, ddir - 2 * np.pi, ddir)
        ddir = np.where(ddir < -np.pi, ddir + 2 * np.pi, ddir)
        ds[f"dirshear_{b}_{t}"] = (("time", "latitude", "longitude"), ddir)
        
        # Temperature lapse rate (°C/km)
        lapse = (ds[f"temp{b}"] - ds[f"temp{t}"]) / (ds[f"height{t}"] - ds[f"height{b}"]) * 1000
        ds[f"lapse_{b}_{t}"] = lapse
        
        # Height and temperature differences
        ds[f"dz_{b}_{t}"] = ds[f"height{t}"] - ds[f"height{b}"]
        ds[f"dt_{b}_{t}"] = ds[f"temp{b}"] - ds[f"temp{t}"]
    
    # ===== AGGREGATE FEATURES (excluding target level) =====
    if len(feature_levels) > 1:
        # Mean atmospheric properties across available levels
        ds["tmean_excl_target"] = ds.t.sel(isobaricInhPa=feature_levels).mean("isobaricInhPa") - 273.15
        ds["wsmean_excl_target"] = ds.wind_speed.sel(isobaricInhPa=feature_levels).mean("isobaricInhPa")
        
        # Atmospheric column characteristics
        if 'q' in ds.data_vars:
            ds["qmean_excl_target"] = ds.q.sel(isobaricInhPa=feature_levels).mean("isobaricInhPa")
        if 'r' in ds.data_vars:
            ds["rmean_excl_target"] = ds.r.sel(isobaricInhPa=feature_levels).mean("isobaricInhPa")
    
    # ===== GEOGRAPHIC AND TEMPORAL FEATURES =====
    # These don't cause leakage as they're independent of target
    ds["lat"] = ds.latitude
    ds["lon"] = ds.longitude
    
    # Time-based features (if time dimension exists)
    if 'time' in ds.dims:
        time_vals = pd.to_datetime(ds.time.values)
        ds["hour"] = (("time",), time_vals.hour)
        ds["day_of_year"] = (("time",), time_vals.dayofyear)
        # Cyclic encoding
        ds["hour_sin"] = np.sin(2 * np.pi * ds["hour"] / 24)
        ds["hour_cos"] = np.cos(2 * np.pi * ds["hour"] / 24)
        ds["doy_sin"] = np.sin(2 * np.pi * ds["day_of_year"] / 365)
        ds["doy_cos"] = np.cos(2 * np.pi * ds["day_of_year"] / 365)
    
    return ds

# ========================================
# STEP 2: TIME-AWARE DATA SPLITTING
# ========================================
def temporal_train_test_split(df, test_ratio=0.2):
    """
    Split time series data temporally (not randomly) to prevent data leakage.
    """
    if 'time' in df.columns:
        df_sorted = df.sort_values('time')
        split_idx = int(len(df_sorted) * (1 - test_ratio))
        train_df = df_sorted.iloc[:split_idx]
        test_df = df_sorted.iloc[split_idx:]
        print(f"  → Temporal split: {len(train_df)} train, {len(test_df)} test samples")
        return train_df, test_df
    else:
        # Fallback to random split if no time column
        print("  → No time column found, using random split")
        from sklearn.model_selection import train_test_split
        return train_test_split(df, test_size=test_ratio, random_state=42)

# ========================================
# STEP 3: LSTM MODEL ARCHITECTURE
# ========================================
def create_lstm_model(input_shape, lstm_units=50, dropout_rate=0.2):
    """
    Create LSTM model for wind speed prediction.
    """
    model = Sequential([
        LSTM(lstm_units, return_sequences=True, input_shape=input_shape),
        Dropout(dropout_rate),
        LSTM(lstm_units // 2, return_sequences=False),
        Dropout(dropout_rate),
        Dense(25, activation='relu'),
        Dense(1, activation='linear')
    ])
    
    model.compile(
        optimizer='adam',
        loss='mse',
        metrics=['mae']
    )
    return model

def prepare_lstm_sequences(X, y, sequence_length=24):
    """
    Prepare sequences for LSTM training.
    """
    X_seq, y_seq = [], []
    for i in range(sequence_length, len(X)):
        X_seq.append(X[i-sequence_length:i])
        y_seq.append(y[i])
    return np.array(X_seq), np.array(y_seq)

# ========================================
# STEP 4: MAIN TRAINING PIPELINE
# ========================================
def train_models_for_all_levels():
    """
    Train separate models for each pressure level.
    """
    print("\n" + "="*40)
    print("STEP 1: LOADING AND PROCESSING DATA")
    print("="*40)
    
    # Load raw data
    print("→ Loading GRIB file...")
    ds_raw = xr.open_dataset(GRIB_PATH, engine="cfgrib")
    print(f"  Raw data shape: {dict(ds_raw.dims)}")
    
    trained_models = {}
    scalers = {}
    
    # Train a model for each pressure level
    for target_level in MODELS_TO_TRAIN:
        print(f"\n{'='*50}")
        print(f"TRAINING MODEL FOR {target_level} hPa")
        print(f"{'='*50}")
        
        # ===== FEATURE ENGINEERING =====
        print("→ Engineering features...")
        ds_feat = engineer_features_safe(ds_raw, target_level)
        
        # Add target variable
        target_name = f"ws{target_level}"
        ds_feat[target_name] = ds_feat.wind_speed.sel(isobaricInhPa=target_level)
        
        # Get feature names (exclude target and intermediate variables)
        exclude_vars = {target_name, 'wind_speed', 'u', 'v', 't', 'z'}
        ALL_FEATURES = [k for k in ds_feat.data_vars.keys() if k not in exclude_vars]
        
        print(f"  → Created {len(ALL_FEATURES)} features")
        
        # ===== CREATE DATAFRAME =====
        print("→ Creating DataFrame...")
        df = (
            ds_feat[ALL_FEATURES + [target_name]]
            .to_dataframe()
            .dropna()
            .reset_index()
        )
        print(f"  → DataFrame shape: {df.shape}")
        
        # ===== LEAKAGE DETECTION =====
        print("→ Checking for data leakage...")
        X_temp = df[ALL_FEATURES]
        y_temp = df[target_name]
        
        correlations = X_temp.corrwith(y_temp).abs().sort_values(ascending=False)
        high_corr = correlations[correlations > 0.95]
        
        if len(high_corr) > 0:
            print(f"  ⚠️  WARNING: Found {len(high_corr)} features with correlation > 0.95:")
            print(high_corr.head())
        else:
            print("  ✅ No obvious data leakage detected")
        
        # ===== TEMPORAL SPLITTING =====
        print("→ Splitting data temporally...")
        train_df, test_df = temporal_train_test_split(df, test_ratio=0.2)
        
        X_train = train_df[ALL_FEATURES]
        y_train = train_df[target_name]
        X_test = test_df[ALL_FEATURES]
        y_test = test_df[target_name]
        
        # ===== FEATURE SCALING =====
        print("→ Scaling features...")
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # ===== MODEL TRAINING =====
        print("→ Training Random Forest...")
        rf_model = RandomForestRegressor(
            n_estimators=100,
            max_depth=15,  # Limit depth to prevent overfitting
            min_samples_split=10,  # Increase minimum samples
            min_samples_leaf=5,    # Increase minimum leaf samples
            oob_score=True,
            n_jobs=-1,
            random_state=42
        )
        
        rf_model.fit(X_train_scaled, y_train)
        
        # ===== EVALUATION =====
        print("→ Evaluating model...")
        y_pred_train = rf_model.predict(X_train_scaled)
        y_pred_test = rf_model.predict(X_test_scaled)
        
        train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
        test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
        train_r2 = r2_score(y_train, y_pred_train)
        test_r2 = r2_score(y_test, y_pred_test)
        train_mae = mean_absolute_error(y_train, y_pred_train)
        test_mae = mean_absolute_error(y_test, y_pred_test)
        
        print(f"\n  📊 MODEL PERFORMANCE FOR {target_level} hPa:")
        print(f"     Training   - RMSE: {train_rmse:.3f}, R²: {train_r2:.3f}, MAE: {train_mae:.3f}")
        print(f"     Testing    - RMSE: {test_rmse:.3f}, R²: {test_r2:.3f}, MAE: {test_mae:.3f}")
        print(f"     OOB Score  - R²: {rf_model.oob_score_:.3f}")
        
        # Check for overfitting
        overfitting_indicator = train_r2 - test_r2
        if overfitting_indicator > 0.1:
            print(f"     ⚠️  Possible overfitting detected (train-test R² gap: {overfitting_indicator:.3f})")
        else:
            print(f"     ✅ Good generalization (train-test R² gap: {overfitting_indicator:.3f})")
        
        # ===== FEATURE IMPORTANCE =====
        print("→ Analyzing feature importance...")
        importances = rf_model.feature_importances_
        feature_importance = pd.DataFrame({
            'feature': ALL_FEATURES,
            'importance': importances
        }).sort_values('importance', ascending=False)
        
        print(f"  Top 5 most important features:")
        for idx, row in feature_importance.head().iterrows():
            print(f"     {row['feature']:20s}: {row['importance']:.4f}")
        
        # ===== SAVE MODEL =====
        model_filename = f"wind_model_{target_level}hPa.pkl"
        scaler_filename = f"scaler_{target_level}hPa.pkl"
        
        joblib.dump(rf_model, model_filename)
        joblib.dump(scaler, scaler_filename)
        
        print(f"  💾 Saved model: {model_filename}")
        print(f"  💾 Saved scaler: {scaler_filename}")
        
        # Store in dictionary for return
        trained_models[target_level] = {
            'model': rf_model,
            'scaler': scaler,
            'features': ALL_FEATURES,
            'performance': {
                'train_rmse': train_rmse,
                'test_rmse': test_rmse,
                'train_r2': train_r2,
                'test_r2': test_r2,
                'oob_r2': rf_model.oob_score_
            },
            'feature_importance': feature_importance
        }
        
        scalers[target_level] = scaler
    
    return trained_models, scalers

# ========================================
# STEP 5: PREDICTION FUNCTION
# ========================================
def predict_wind_along_path(models, scalers, path_points):
    """
    Predict wind speeds along a UAV flight path.
    
    path_points: List of dictionaries with keys:
        - 'time': timestamp
        - 'latitude': latitude in degrees
        - 'longitude': longitude in degrees  
        - 'altitude_m': altitude in meters
    """
    print(f"\n→ Predicting wind for {len(path_points)} path points...")
    
    predictions = []
    
    for point in path_points:
        # Here you would:
        # 1. Load weather data for point's time/location
        # 2. Extract features using the same feature engineering
        # 3. Apply appropriate model based on altitude
        # 4. Interpolate between pressure levels if needed
        
        # For demonstration, we'll show the structure
        altitude_m = point['altitude_m']
        
        # Convert altitude to approximate pressure level
        # (This is simplified - in reality you'd use the geopotential height fields)
        if altitude_m < 100:
            target_level = 1000
        elif altitude_m < 600:
            target_level = 950
        elif altitude_m < 1200:
            target_level = 900
        else:
            target_level = 850
        
        # Get the appropriate model
        if target_level in models:
            model_info = models[target_level]
            # prediction = model_info['model'].predict(scaled_features)
            # For now, just return the target level used
            predictions.append({
                'point': point,
                'model_used': target_level,
                'predicted_wind': None  # Would be actual prediction
            })
    
    return predictions

# ========================================
# MAIN EXECUTION
# ========================================
if __name__ == "__main__":
    print("Starting UAV Wind Prediction Pipeline...")
    
    # Train models for all pressure levels
    trained_models, scalers = train_models_for_all_levels()
    
    print(f"\n{'='*60}")
    print("TRAINING COMPLETED!")
    print(f"{'='*60}")
    print(f"✅ Successfully trained {len(trained_models)} models")
    print(f"✅ Models available for pressure levels: {list(trained_models.keys())} hPa")
    
    # Summary of all models
    print(f"\n📊 OVERALL PERFORMANCE SUMMARY:")
    print(f"{'Level (hPa)':<12} {'Test R²':<10} {'Test RMSE':<12} {'Overfitting':<12}")
    print("-" * 50)
    
    for level, info in trained_models.items():
        perf = info['performance']
        overfitting = perf['train_r2'] - perf['test_r2']
        overfitting_status = "⚠️ High" if overfitting > 0.1 else "✅ Low"
        
        print(f"{level:<12} {perf['test_r2']:<10.3f} {perf['test_rmse']:<12.3f} {overfitting_status}")
    
    print(f"\n🎯 Next steps:")
    print(f"   1. Use predict_wind_along_path() function for UAV route planning")
    print(f"   2. Load models with joblib.load() for deployment")
    print(f"   3. Integrate with real-time weather data feeds")
    
    print(f"\n{'='*60}")
    print("PIPELINE READY FOR DEPLOYMENT!")
    print(f"{'='*60}")

UAV WIND PREDICTION PIPELINE - MULTI-LEVEL APPROACH
Training models for pressure levels: [1000, 950, 900, 850] hPa
Data source: C:/Users/Victus/Downloads/953c78511a142e4b81be9c53038b62a5.grib
Starting UAV Wind Prediction Pipeline...

STEP 1: LOADING AND PROCESSING DATA
→ Loading GRIB file...
  Raw data shape: {'time': 1464, 'isobaricInhPa': 4, 'latitude': 15, 'longitude': 17}

TRAINING MODEL FOR 1000 hPa
→ Engineering features...
  → Engineering features for 1000 hPa target (excluding 1000 hPa data)
  → Using feature levels: [950, 900, 850]
  → Computing inter-level features for 3 level pairs
  → Created 69 features
→ Creating DataFrame...
  → DataFrame shape: (1493280, 77)
→ Checking for data leakage...
  ✅ No obvious data leakage detected
→ Splitting data temporally...
  → Temporal split: 1194624 train, 298656 test samples
→ Scaling features...
→ Training Random Forest...
→ Evaluating model...

  📊 MODEL PERFORMANCE FOR 1000 hPa:
     Training   - RMSE: 0.210, R²: 0.964, MAE: 0.145
 

In [17]:

parser = create_cli()
    # <-- change this line
args, unknown = parser.parse_known_args()
if unknown:
    logger.debug("Ignoring unknown args: %s", unknown)
